In [1]:
#MultiClass classifier. Predicts one attribute as one of many labels rather than just 2 possible labels.
#This assumes there is only one right answer. We could do multilabel classification and present multiple city solutions to the user?
#https://scikit-learn.org/stable/modules/multiclass.html    a look at different kinds of models

""" method of going through multiple models
 https://medium.com/analytics-vidhya/testing-multiple-machine-learning-models-at-once-without-getting-a-headache-5aefb0e7df03    simple good
 https://github.com/j-planet/machine-learning-big-loop   more advanced, but better. (get parameters.py from this, and the cell for running multiple things"""

"""https://stackoverflow.com/questions/23045318/grid-search-over-multiple-classifiers
Smallest, simple to

https://www.codegrepper.com/code-examples/whatever/gridsearchcv+with+multiple+models
Straightforward

https://www.davidsbatista.net/blog/2018/02/23/model_optimization/
Large, hard to understand, but powerful

https://github.com/davidsbatista/machine-learning-notebooks/blob/master/hyperparameter-across-models.ipynb
Large, hard to understand, but powerful"""


#to do
# find best testing method
# get best parameters

#to ask
#why do his cells not work

# Import necessary modules from repository
from model.data.pipeline import sql_strings as sql, utils as ut
import numpy as np
import pandas as pd


In [2]:
#dataframe of all monthly data
df = ut.load_sql_as_df('SELECT * From MonthlyDataModel;')

#dataframe of month averages to fill in bad values
monthAvg = ut.load_sql_as_df('SELECT * From MonthlyAverages;')


In [3]:
# Getting features and choosing labels. Fill NaN spots with NaN text to manipulate them
labels = df["state"]
features = df.drop(['region', 'latitude', 'longitude', 'elevation', 'region'], axis=1)
features = features.fillna('NaN')

# Changing columns in to months to help compare
featSer = features["month"].to_numpy()
prcpIntM = monthAvg["prcpInt"].to_numpy()
prcpFreqM = monthAvg["prcpFreq"].to_numpy()
temp_max_normalM = monthAvg["temp_max_normal"].to_numpy()
temp_min_normalM = monthAvg["temp_min_normal"].to_numpy()
snow_depthM = monthAvg["snowInt"].to_numpy()
snow_daysM = monthAvg["snowFreq"].to_numpy()
cloudsM = monthAvg["clouds"].to_numpy()

prcp_normRay = features["prcp_normal"].to_numpy()
prcp_days_tRay = features["prcp_days_t"].to_numpy()
temp_max_normalRay = features["temp_max_normal"].to_numpy()
temp_min_normalRay = features["temp_min_normal"].to_numpy()
snow_depthRay = features["snow_depth_days"].to_numpy()
snow_daysRay = features["snow_days_t"].to_numpy()
cloudsRay = features["clouds"].to_numpy()

#replacing bad values
for i in range(len(featSer)):
    if prcp_normRay[i] == 'NaN' or prcp_normRay[i] == -9999 or prcp_normRay[i] == -8888 or prcp_normRay[i] == -7777 or prcp_normRay[i] == -6666 or prcp_normRay[i] == -5555:
        prcp_normRay[i] = prcpIntM[featSer[i]-1]
    if prcp_days_tRay[i] == 'NaN' or prcp_days_tRay[i] == -9999 or prcp_days_tRay[i] == -8888 or prcp_days_tRay[i] == -7777 or prcp_days_tRay[i] == -6666 or prcp_days_tRay[i] == -5555:
        prcp_days_tRay[i] = prcpFreqM[featSer[i]-1]
    if temp_max_normalRay[i] == 'NaN' or temp_max_normalRay[i] == -9999 or temp_max_normalRay[i] == -8888 or temp_max_normalRay[i] == -7777 or temp_max_normalRay[i] == -6666 or temp_max_normalRay[i] == -5555:
        temp_max_normalRay[i] = temp_max_normalM[featSer[i]-1]
    if temp_min_normalRay[i] == 'NaN' or temp_min_normalRay[i] == -9999 or temp_min_normalRay[i] == -8888 or temp_min_normalRay[i] == -7777 or temp_min_normalRay[i] == -6666 or temp_min_normalRay[i] == -5555:
        temp_min_normalRay[i] = temp_min_normalM[featSer[i]-1]
    if snow_depthRay[i] == 'NaN' or snow_depthRay[i] == -9999 or snow_depthRay[i] == -8888 or snow_depthRay[i] == -7777 or snow_depthRay[i] == -6666 or snow_depthRay[i] == -5555:
        snow_depthRay[i] = snow_depthM[featSer[i]-1]
    if snow_daysRay[i] == 'NaN' or snow_daysRay[i] == -9999 or snow_daysRay[i] == -8888 or snow_daysRay[i] == -7777 or snow_daysRay[i] == -6666 or snow_daysRay[i] == -5555:
        snow_daysRay[i] = snow_daysM[featSer[i]-1]
    if cloudsRay[i] == 'NaN' or cloudsRay[i] == -9999 or cloudsRay[i] == -8888 or cloudsRay[i] == -7777 or cloudsRay[i] == -6666 or cloudsRay[i] == -5555:
        cloudsRay[i] = cloudsM[featSer[i]-1]

# Recreating filled dataset
features = pd.DataFrame({'prcp_normRay' : prcp_normRay, 'prcp_days_tRay' : prcp_days_tRay, "temp_max_normalRay" : temp_max_normalRay, "temp_min_normalRay" : temp_min_normalRay, "snow_depthRay" : snow_depthRay, "snow_daysRay" : snow_daysRay, "cloudsRay" : cloudsRay})


In [4]:
features

,prcp_normRay,prcp_days_tRay,temp_max_normalRay,temp_min_normalRay,snow_depthRay,snow_daysRay,cloudsRay
0,5.3,69.0,41.924644,21.303576,0.0,0.0,55.667977
1,5.07,65.0,45.985372,24.106816,0.0,0.0,55.013862
2,5.8,62.0,54.495133,31.252366,-309.95931,-297.336583,52.051446
3,3.77,48.0,64.407005,39.586609,0.0,0.0,49.932187
4,3.82,48.0,73.406937,48.77541,0.0,0.0,48.941521
...,...,...,...,...,...,...,...
117022,3.22,25.909684,85.4,62.7,-6.026158,-32.307496,40.020323
117023,3.04,46.810539,77.8,53.6,-156.429374,-337.711363,41.015647
117024,2.04,47.409751,65.2,40.6,-553.695795,-656.829311,45.018233
117025,1.41,49.541899,49.7,28.0,-244.419299,-363.569282,51.45818


In [5]:
from sklearn.preprocessing import StandardScaler

#Scalling all data
scaler = StandardScaler()
featPrepped = scaler.fit_transform(features)
featPrepped

array([[ 0.91107733,  0.82026143, -1.25009436, ...,  0.54633954,
         0.56557525,  1.15810527],
       [ 0.82384972,  0.64959163, -1.02602689, ...,  0.54633954,
         0.56557525,  1.0542527 ],
       [ 1.10070256,  0.52158929, -0.55646573, ..., -1.26585409,
        -1.02904868,  0.58391578],
       ...,
       [-0.3252792 , -0.1009394 ,  0.03421949, ..., -2.69087244,
        -2.95701765, -0.53273362],
       [-0.564207  , -0.00996612, -0.82105698, ..., -0.88267092,
        -1.38425639,  0.48972407],
       [-0.74624723,  0.01113871, -1.52734981, ..., -0.84865873,
        -0.99557283,  1.16723128]])

In [6]:
from sklearn.model_selection import train_test_split

feat_train, feat_test, lab_train, lab_test = train_test_split(featPrepped, labels, random_state=42)
# stratify by city for separating training and testing. stratified sampling

In [7]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(feat_train, lab_train)

print(clf.score(feat_test, lab_test))  #This automatically makes predictions on feat_test


0.40855179956933385


In [8]:

#Next 3 cells are simons and do not work

In [9]:
"""import numpy as np
TOP = 5
TMP = 62
CITIES = labels.unique()

predictions_list = []
for i in range(len(df)):
    row = df.iloc[i].sort_values(ascending=False)[:TOP]
    rowDf = row.to_frame().reset_index()
    #print([[CITIES[int(x[0])], x[1]] for x in rowDf.to_numpy()])
    vals = np.insert(np.array([[CITIES[int(x[0])], x[1]] for x in rowDf.to_numpy()]), 0, lab_test.iloc[i])
    predictions_list.append(vals)

cols = ['true']
for i in range(TOP):
    cols.append('city{x}'.format(x=i + 1))
    cols.append('prob{x}'.format(x=i + 1))

preds = pd.DataFrame(predictions_list,
                     columns=cols)
preds
"""

TypeError: '>' not supported between instances of 'numpy.ndarray' and 'str'

In [ ]:
"""count = 0
for i in range(len(preds)):
    if preds.iloc[i]['true'] == preds.iloc[i]['city1']:
        count += 1
        #print('{i}: {t} and {p}'.format(i=i, t=preds['true'][i], p=preds['city1'][i]))

print('Accuracy: {r}/{t}, {p}%'.format(r=count, t=len(preds), p=round(100*count/len(preds))))"""

In [ ]:
"""CITY = 'Denver'

for i in range(len(preds)):
    row = preds.iloc[i]
    if row['true'] == CITY and row['city1'] != CITY:
        print(row['city1'])
CITIES"""

In [17]:
from sklearn.svm import SVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid, RadiusNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

log_reg_params = [{"C": 0.01}, {"C": 0.1}, {"C": 1}, {"C": 10}]
dec_tree_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
rand_for_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
kneighbors_params = [{"n_neighbors": 3}, {"n_neighbors": 5}]
naive_bayes_params = [{}]
svc_params = [{"C": 0.01}, {"C": 0.1}, {"C": 1}, {"C": 10}]

modelclasses = [
    ["GradientBoostingClf", GradientBoostingClassifier()],
    ["ExtraTreeClf", ExtraTreesClassifier()],
    ["decision tree", DecisionTreeClassifier()],
    ["random forest", RandomForestClassifier()],
    ["k neighbors", KNeighborsClassifier()],
    ["NearestCentroid", NearestCentroid()],
    #["RadiusNeighbor", RadiusNeighborsClassifier()],
    ["GaussianProcess", GaussianProcessClassifier()],
    ["naive bayes Gaussian NB", GaussianNB()],
    ["naive bayes Bernoulli NB", BernoulliNB()],
    ["support vector machines", SVC()],
    ["Nu support vector machines", NuSVC()],
    ["NN MPL", MLPClassifier()]
]

insights = []
for modelname, model in modelclasses:
    try:
        model.fit(feat_train, lab_train)
        score = model.score(feat_test, lab_test)
        insights.append((modelname, model, score))
    except:
        print(modelname, " did not work \n")


insights.sort(key=lambda x: x[-1], reverse=True)
for modelname, model, score in insights:
    print(modelname, score)





GaussianProcess  did not work 

Nu support vector machines  did not work 

random forest 0.40834672044297093
ExtraTreeClf 0.40028027480602935
k neighbors 0.361349420651468
NN MPL 0.337765321119732
GradientBoostingClf 0.33509929247701403
decision tree 0.3122329698875483
support vector machines 0.29975732303380387
naive bayes Gaussian NB 0.1117339440133985
naive bayes Bernoulli NB 0.07755408961957821
NearestCentroid 0.06360870902689955


c:\users\trenton\documents\programming\ml-team3\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [12]:
from sklearn.model_selection import GridSearchCV

GBR = GradientBoostingRegressor()

para_grid = [{
    "max_depth" : np.linspace(1, 15, 5, dtype = int),
    "n_estimators" : np.linspace(1, 200, 10, dtype = int),
    "learning_rate" :  np.linspace(0.05, 2, 10).round(2)
}]

grid_search = GridSearchCV(GBR, para_grid, cv = 3)
grid_search.fit(X_train, z_train)

print(grid_search.best_params_)